# Energy and carbon accounting -- footprint calculations

This notebook compiles all notebooks relevant to the analysis. For computation and detailed information on data fetching and cleaning, see the referenced notebooks.
* Contact person: Joakim Blix Prestmo
* Date: 19th May 2021
* Data sources: See details in code (it is a combination of intenal and external sources)
* This master file needs inputs from six noteboks - all attached in this repository

In [5]:
import pandas as pd

## Scope 1

In [13]:
# assumption, 5 rides a week, average of 25 km per ride 
km_year = 5*48*25
# emission data for WV Golf: https://www.vegvesen.no/kjoretoy/Kjop+og+salg/Kj%C3%B8ret%C3%B8yopplysninger/sjekk-andres-kjoretoy
emission_km_car = 0.156 
emission_car_scope1 = km_year * emission_km_car


In [14]:
df_S1 = pd.DataFrame(columns=['Category', 'Description', 'Consumption', 'Unit', "Energy (MWh)", "Emissions (tCO2e)", "Emissions (distribution)"])
df_S1.loc[0] = ['Owned company car', ' ', km_year, 'pkm', ' ', emission_car_scope1, ' ' ]

In [15]:
df_S1

,Category,Description,Consumption,Unit,Energy (MWh),Emissions (tCO2e),Emissions (distribution)
0,Owned company car,,6000,pkm,,936.0,


## Scope 2

In [1]:
%run S2_total_2020.ipynb

Total CO2 emission by location-based method = 32651 kg CO2e
Total CO2 emission by the market-based method = 287809 kg CO2e


In [2]:
df_S2 = pd.DataFrame(columns = ["Category", "Description", "Consumption", "Unit", "Energy (MWh)", "Emissions (tCO2e)", "Emissions (distribution)"])

df_S2.loc[0] = ["Electricity nordic mix", " ", df_electricity[~df_electricity.index.str.contains("Spain") & ~df_electricity.index.str.contains("Total")]["Consumption [kWh]"].sum(), "kWh", df_electricity[~df_electricity.index.str.contains("Spain") & ~df_electricity.index.str.contains("Total")]["Consumption [kWh]"].sum() / 1000, df_electricity[~df_electricity.index.str.contains("Spain") & ~df_electricity.index.str.contains("Total")]["Calculated emission [kg CO2e]"].sum() / 1000, np.nan]
df_S2.loc[1] = ["Electricity Spain", " ", df_electricity[df_electricity.index.str.contains("Spain")]["Consumption [kWh]"].sum(), "kWh", df_electricity[df_electricity.index.str.contains("Spain")]["Consumption [kWh]"].sum() / 1000, df_electricity[df_electricity.index.str.contains("Spain")]["Calculated emission [kg CO2e]"].sum() / 1000, np.nan]
df_S2.loc[2] = ["District heating: Trondheim", " ", df_distHeat['Consumption [kWh]'][0], "kWh", df_distHeat['Consumption [kWh]'][0] / 1000, df_distHeat['Calculated emission [kg CO2e]'][0] / 1000, np.nan]

#df_S2.loc[3] = ["Scope 2 total", " ", df_S2['Consumption'].sum(), "kWh", df_S2['Consumption'].sum() / 1000, df_S2['Emissions (tCO2e)'].sum(), np.nan]

df_S2['Consumption'] = df_S2['Consumption'].astype(int)
df_S2

,Category,Description,Consumption,Unit,Energy (MWh),Emissions (tCO2e),Emissions (distribution)
0,Electricity nordic mix,,833489,kWh,833.489,32.506071,NaN
1,Electricity Spain,,538,kWh,0.538,0.145260,NaN
2,District heating: Trondheim,,83928,kWh,83.928,4.347470,NaN


## Scope 3

### Upstream activities

#### Business travel

In [3]:
%run S3_businessTravel_2020.ipynb

Business travel by train: 
verdict_train: 


,Distance traveled (pkm),Total CO2 equivalents (tonnes)
Train,10977.0,0.14


Business travel by road:
verdict_road: 


,Distance traveled (pkm),Total CO2 equivalents (tonnes)
Private car,12084,1.32
Taxi,725,0.08
Bus,1280,0.02
Road total,14089,1.42


Business air travel:
verdict_flight:


,Distance traveled (pkm),Total CO2 equivalents (tonnes)
Flight: Areas,215788,104.7


#### Employee commuting

In [4]:
%run "S3_commuting_2020.ipynb"

Kg CO2 per km =  0.7452183995081625
Co2 utslipp per km per reisende = 0.015855710627833246
Utslipp fra buss i året = 4.56 tCO2e
Utslipp fra bilkjøring i året = 3.07 tCO2
8.0
Utslipp fra EL bilkjøring i året = 0.1 tCO2
Employee commuting
df_verdict_commute:


,Distanse [pkm],"Utslipp, CO2 [tonn]"
Bil,21652,3.07
Buss,287433,4.56
EL-bil,28869,0.10


### Waste

In [23]:
%run "S3_waste_2020.ipynb"

Waste, Trondheim
df_verdict_waste:


,Avfall [kg],"Utslipp, CO2 [tonn]"
Kontorpapir til gjenvinning,870.0000,NaN
"Papir til makulering, til gjenvinning",612.0000,NaN
"EE-avfall, gjenvinnes",80.0000,NaN
Restavfall til forbrenning,3399.3960,1.71
"Metaller, til gjenvinning",33.1110,NaN
"Glass, til gjennvinning",515.0600,NaN
Plastikk folie til gjenvinning,45.9875,NaN
Matavfall til gjenvinning,114.0490,NaN
Trevirke til gjenvinning,217.0610,NaN
Papp og papir til gjenvinning,1044.8360,NaN


In [24]:
df_S3_upstream = pd.DataFrame(columns = ["Category", "Description", "Consumption", "Unit", "Energy (MWh)", "Emissions (tCO2e)", "Emissions (distribution)"])

df_S3_upstream.loc[0, "Category"] = "Business travel"
df_S3_upstream.loc[1] = ["Train", " ", df_total.loc["Train", "Distance traveled (pkm)"], "pkm", np.nan, df_total.loc["Train", "Total CO2 equivalents (tonnes)"], np.nan]
df_S3_upstream.loc[2] = ["Road", " ", df_total.loc["Road total", "Distance traveled (pkm)"], "pkm", np.nan, df_total.loc["Road total", "Total CO2 equivalents (tonnes)"], np.nan]

df_S3_upstream.loc[3, "Category"] = "Air travel"
df_S3_upstream.loc[4] = ["Total flights", " ", df_total.loc["Flight: Areas", "Distance traveled (pkm)"], "pkm", np.nan, df_total.loc["Flight: Areas", "Total CO2 equivalents (tonnes)"], np.nan]

# df_S3_upstream.loc[4] = ["Domestic flights", " ", df_total.loc["Flight: Domestic", "Distance traveled (pkm)"], "pkm", np.nan, df_total.loc["Flight: Domestic", "Total CO2 equivalents (tonnes)"], np.nan]
# df_S3_upstream.loc[5] = ["Nordic flights", " ", df_total.loc["Flight: Nordic", "Distance traveled (pkm)"], "pkm", np.nan, df_total.loc["Flight: Nordic", "Total CO2 equivalents (tonnes)"], np.nan]
# df_S3_upstream.loc[6] = ["Europe flights", " ", df_total.loc["Flight: Europe", "Distance traveled (pkm)"], "pkm", np.nan, df_total.loc["Flight: Europe", "Total CO2 equivalents (tonnes)"], np.nan]

df_S3_upstream.loc[7, "Category"] = "Commuting"
df_S3_upstream.loc[8] = ["Car", " ", df_verdict_commute.loc["Bil", "Distanse [pkm]"], "km", np.nan, df_verdict_commute.loc["Bil", "Utslipp, CO2 [tonn]"], np.nan]
df_S3_upstream.loc[9] = ["Bus", " ", df_verdict_commute.loc["Buss", "Distanse [pkm]"], "pkm", np.nan, df_verdict_commute.loc["Buss", "Utslipp, CO2 [tonn]"], np.nan]

df_S3_upstream.loc[10, "Category"] = 'Waste'
df_S3_upstream.loc[11] = ['Mixed incinerated', ' ', df_verdict_waste.loc['Restavfall til forbrenning', 'Avfall [kg]'], 'kg', np.nan, df_verdict_waste.loc['Restavfall til forbrenning', 'Utslipp, CO2 [tonn]'],np.nan]

df_S3_upstream = df_S3_upstream.replace(np.nan, '', regex=True)
df_S3_upstream

,Category,Description,Consumption,Unit,Energy (MWh),Emissions (tCO2e),Emissions (distribution)
0,Business travel,,,,,,
1,Train,,10977,pkm,,0.14,
2,Road,,14089,pkm,,1.42,
3,Air travel,,,,,,
4,Total flights,,215788,pkm,,104.7,
7,Commuting,,,,,,
8,Car,,21652,km,,3.07,
9,Bus,,287433,pkm,,4.56,
10,Waste,,,,,,
11,Mixed incinerated,,3399.4,kg,,1.71,


                                       Avfall [kg]  Utslipp, CO2 [tonn]
Kontorpapir til gjenvinning               870.0000                  NaN
Papir til makulering, til gjenvinning     612.0000                  NaN
EE-avfall, gjenvinnes                      80.0000                  NaN
Restavfall til forbrenning               3399.3960                  NaN
Metaller, til gjenvinning                  33.1110                  NaN
Glass, til gjennvinning                   515.0600                  NaN
Plastikk folie til gjenvinning             45.9875                  NaN
Matavfall til gjenvinning                 114.0490                  NaN
Trevirke til gjenvinning                  217.0610                  NaN
Papp og papir til gjenvinning            1044.8360                  NaN
Farlig avfall til gjenvinning              45.9875                  NaN


### Downstream activities

#### Business market
Emissions due to electricity consumption of comercial buildings (næringsbygg)

In [2]:
%run S3_downsream_businessMarket.ipynb

#### Private market

In [1]:
%run S3_downstream_privateMarket.ipynb

In [8]:
df_S3_downstream = pd.DataFrame(columns = ["Category", "Description", "Consumption", "Unit", "Energy (MWh)", "Emissions (tCO2e)", "Emissions (distribution)"])

df_S3_downstream.loc[0, "Category"] = "Downstream activities"

# Emission factor is from Nordpool Nordic Mix average,  tCO2 / GWh
downstream_emission_factor = 39

PM_energy_consumption = df_BNB_pm_energygrades.loc["Total","Small_house_GWh"] + df_BNB_pm_energygrades.loc["Total","Apartment_GWh"]
BM_energy_consumption = df_tenants_grouped["net_energy_consumtpion_GWh"].sum()

df_S3_downstream.loc[1] = ["Private market", "Consumed electricity from residencies funded by BN Bank's loans", PM_energy_consumption, "GWh", PM_energy_consumption * 1000, PM_energy_consumption*downstream_emission_factor, np.nan]
df_S3_downstream.loc[2] = ["Business market", "Consumed electricity from businesses funded by BN Bank's loans", BM_energy_consumption, "GWh", BM_energy_consumption * 1000, BM_energy_consumption*downstream_emission_factor, np.nan]

df_S3_downstream = df_S3_downstream.replace(np.nan, '', regex=True)
df_S3_downstream

,Category,Description,Consumption,Unit,Energy (MWh),Emissions (tCO2e),Emissions (distribution)
0,Downstream activities,,,,,,
1,Private market,Consumed electricity from residencies funded b...,213.03,GWh,213030,8308.17,
2,Business market,Consumed electricity from businesses funded by...,136.486,GWh,136486,5322.97,


# Summary

In [25]:
df_summary = pd.concat([df_S1, df_S2, df_S3_upstream, df_S3_downstream])

# Convert emissions to float values
df_summary["Emissions (tCO2e)"][df_summary["Emissions (tCO2e)"] == ""] = np.nan
#df_summary['Emissions (tCO2e)'] = pd.to_numeric(df_summary['Emissions (tCO2e)'], errors='coerce')
df_summary['Emissions (tCO2e)'].astype(float)


# Compute distribution of emissions. Not including downstream activity
BNB_tot_emissions = df_summary[(df_summary["Emissions (tCO2e)"] != "") & (df_summary["Unit"] != "GWh")]["Emissions (tCO2e)"].sum()
df_summary["Emissions (distribution)"][(df_summary["Emissions (tCO2e)"] != "") & (df_summary["Unit"] != "GWh")] = df_summary["Emissions (tCO2e)"][(df_summary["Emissions (tCO2e)"] != "") & (df_summary["Unit"] != "GWh")] / BNB_tot_emissions * 100

df_summary.reset_index(drop=True, inplace=True)

# Sums up the distributions and adds them to the lines Business travel, Air travel and Commuting
df_summary.iloc[3]["Emissions (distribution)"] = df_summary[(df_summary["Category"] == "Train") | (df_summary["Category"] == "Road")]["Emissions (distribution)"].sum()
df_summary.iloc[6]["Emissions (distribution)"] = df_summary[(df_summary["Category"] == "Total flights") ]["Emissions (distribution)"].sum()
df_summary.iloc[10]["Emissions (distribution)"] = df_summary[(df_summary["Category"] == "Car") | (df_summary["Category"] == "Bus")]["Emissions (distribution)"].sum()


df_summary

,Category,Description,Consumption,Unit,Energy (MWh),Emissions (tCO2e),Emissions (distribution)
0,Electricity nordic mix,,833489,kWh,833.489,32.5061,21.3017
1,Electricity Spain,,538,kWh,0.538,0.14526,0.0951908
2,District heating: Trondheim,,83928,kWh,83.928,4.34747,2.84895
3,Business travel,,,,,NaN,1.02229
4,Train,,10977,pkm,,0.14,0.0917438
5,Road,,14089,pkm,,1.42,0.930545
6,Air travel,,,,,NaN,68.6113
7,Total flights,,215788,pkm,,104.7,68.6113
8,Commuting,,,,,NaN,NaN
9,Car,,21652,km,,3.07,2.01181


## Write directly to a MS Word document

In [ ]:
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH

In [ ]:
doc = Document()

doc.add_heading("Energy and GHG emissions")

main_table = doc.add_table(1, 7, style = 'Table Grid')

# Make header row
table_header = ["Category", "Description", "Consumption", "Unit", "Energy (MWh)", "Emissions (tCO2e)", "Emissions (distribution)"]
for header in table_header:
    main_table.cell(0, table_header.index(header)).text = header

# Add Scope 1
main_table.add_row().cells[0].text = "Scope 1"

# Add Scope 2
main_table.add_row().cells[0].text = "Scope 2"

# Add contents from scope 2 dataframe
for i in range(df_S2.shape[0]):
    main_table.add_row()
    for j in range(df_S2.shape[-1]):
        main_table.cell(len(main_table.rows)-1, j).text = str(df_S2.values[i,j])

# Add Scope 3
main_table.add_row().cells[0].text = "Scope 3"
        
# Add contents from scope 3 upstream dataframe
for i in range(df_S3_upstream.shape[0]):
    main_table.add_row()
    for j in range(df_S3_upstream.shape[-1]):
        main_table.cell(len(main_table.rows)-1, j).text = str(df_S3_upstream.values[i,j])

# Add contents from scope 3 downstream dataframe
for i in range(df_S3_downstream.shape[0]):
    main_table.add_row()
    for j in range(df_S3_downstream.shape[-1]):
        main_table.cell(len(main_table.rows)-1, j).text = str(df_S3_downstream.values[i,j])

In [ ]:
doc.save('ghg_carbon_accounting.docx')